# Part 8 - Federated Learning on MNIST using a CNN

We show here how  simple it is to switch to Pytorch models to syft models so as to perform Federated Learning, by using directly the canonical example of training a CNN on MNIST which provided by PyTorch [here](https://github.com/pytorch/examples/blob/master/mnist/main.py). We only changed the code to run in a notebook and we mention all the changed made which enable Federated Learning.

**Roadmap of future improvements:**

 _See the #TODO for details about the following issues:_
 - support **momentum args in optim SGD** #1896
 - silence the Warning in log_softmax -> needs a hook_kwargs


Authors:
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

### Imports and model specifications

First we make the official imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

And than those specific to PySyft. In particular we define remote workers `alice` and `bob`.

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

We define the setting of the learning task

In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Data loading and sending to workers
We first load the data and transform and training Dataset into a Federated Dataset split across the workers using the `.federate` method. This federated dataset is now given to a Federated DataLoader. The test dataset remains unchanged.

In [4]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

Scanning and sending data to bob, alice...
Done!


### CNN specification
Here we use exactly the same CNN as in the official example.

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1) #TODO: shouldn't raise warning

### Define the train and test functions
For the train function, because the data batches are distributed across `alice` and `bob`, you need for each batch to send the model to the right location, then you perform all the operations remotely with the same syntax than if you were doing local PyTorch. When you're done, you get back the model updated and the loss to look for improvement.

In [6]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

The test function does not change!

In [7]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Launch the training !
The traning is done as usual!

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")


/Users/ryffel/Documents/Code/PySyft/syft/frameworks/torch/tensors/interpreters/native.py:216: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  response = eval(cmd)(*args)


Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.273475
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.216173
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.139428
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.053060
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896600
Train Epoch: 1 [4480/60032 (7%)]	Loss: 1.917216
Train Epoch: 1 [5120/60032 (9%)]	Loss: 1.655011
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.440342
Train Epoch: 1 [6400/60032 (11%)]	Loss: 1.231290
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.983513
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.867222
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.890978
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.862243
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.654238
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.587152
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.693086
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.593367
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.531858
Train Epoch: 1 [12800/60032 (

Train Epoch: 2 [44800/60032 (75%)]	Loss: 0.100407
Train Epoch: 2 [45440/60032 (76%)]	Loss: 0.106600
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.070548
Train Epoch: 2 [46720/60032 (78%)]	Loss: 0.059123
Train Epoch: 2 [47360/60032 (79%)]	Loss: 0.185734
Train Epoch: 2 [48000/60032 (80%)]	Loss: 0.067709
Train Epoch: 2 [48640/60032 (81%)]	Loss: 0.090186
Train Epoch: 2 [49280/60032 (82%)]	Loss: 0.040856
Train Epoch: 2 [49920/60032 (83%)]	Loss: 0.133938
Train Epoch: 2 [50560/60032 (84%)]	Loss: 0.067237
Train Epoch: 2 [51200/60032 (85%)]	Loss: 0.046040
Train Epoch: 2 [51840/60032 (86%)]	Loss: 0.037075
Train Epoch: 2 [52480/60032 (87%)]	Loss: 0.226975
Train Epoch: 2 [53120/60032 (88%)]	Loss: 0.054416
Train Epoch: 2 [53760/60032 (90%)]	Loss: 0.064413
Train Epoch: 2 [54400/60032 (91%)]	Loss: 0.050753
Train Epoch: 2 [55040/60032 (92%)]	Loss: 0.117795
Train Epoch: 2 [55680/60032 (93%)]	Loss: 0.148967
Train Epoch: 2 [56320/60032 (94%)]	Loss: 0.109889
Train Epoch: 2 [56960/60032 (95%)]	Loss: 0.041369


Train Epoch: 4 [28800/60032 (48%)]	Loss: 0.083896
Train Epoch: 4 [29440/60032 (49%)]	Loss: 0.109251
Train Epoch: 4 [30080/60032 (50%)]	Loss: 0.019829
Train Epoch: 4 [30720/60032 (51%)]	Loss: 0.015455
Train Epoch: 4 [31360/60032 (52%)]	Loss: 0.223642
Train Epoch: 4 [32000/60032 (53%)]	Loss: 0.036851
Train Epoch: 4 [32640/60032 (54%)]	Loss: 0.211329
Train Epoch: 4 [33280/60032 (55%)]	Loss: 0.099179
Train Epoch: 4 [33920/60032 (57%)]	Loss: 0.030599
Train Epoch: 4 [34560/60032 (58%)]	Loss: 0.031227
Train Epoch: 4 [35200/60032 (59%)]	Loss: 0.057524
Train Epoch: 4 [35840/60032 (60%)]	Loss: 0.108883
Train Epoch: 4 [36480/60032 (61%)]	Loss: 0.018255
Train Epoch: 4 [37120/60032 (62%)]	Loss: 0.070820
Train Epoch: 4 [37760/60032 (63%)]	Loss: 0.105461
Train Epoch: 4 [38400/60032 (64%)]	Loss: 0.051815
Train Epoch: 4 [39040/60032 (65%)]	Loss: 0.015466
Train Epoch: 4 [39680/60032 (66%)]	Loss: 0.068018
Train Epoch: 4 [40320/60032 (67%)]	Loss: 0.065431
Train Epoch: 4 [40960/60032 (68%)]	Loss: 0.015747


Train Epoch: 6 [12800/60032 (21%)]	Loss: 0.041316
Train Epoch: 6 [13440/60032 (22%)]	Loss: 0.065919
Train Epoch: 6 [14080/60032 (23%)]	Loss: 0.009189
Train Epoch: 6 [14720/60032 (25%)]	Loss: 0.037372
Train Epoch: 6 [15360/60032 (26%)]	Loss: 0.099166
Train Epoch: 6 [16000/60032 (27%)]	Loss: 0.069649
Train Epoch: 6 [16640/60032 (28%)]	Loss: 0.053255
Train Epoch: 6 [17280/60032 (29%)]	Loss: 0.084463
Train Epoch: 6 [17920/60032 (30%)]	Loss: 0.005429
Train Epoch: 6 [18560/60032 (31%)]	Loss: 0.104761
Train Epoch: 6 [19200/60032 (32%)]	Loss: 0.068382
Train Epoch: 6 [19840/60032 (33%)]	Loss: 0.052001
Train Epoch: 6 [20480/60032 (34%)]	Loss: 0.029537
Train Epoch: 6 [21120/60032 (35%)]	Loss: 0.082311
Train Epoch: 6 [21760/60032 (36%)]	Loss: 0.069474
Train Epoch: 6 [22400/60032 (37%)]	Loss: 0.101588
Train Epoch: 6 [23040/60032 (38%)]	Loss: 0.043048
Train Epoch: 6 [23680/60032 (39%)]	Loss: 0.080579
Train Epoch: 6 [24320/60032 (41%)]	Loss: 0.003213
Train Epoch: 6 [24960/60032 (42%)]	Loss: 0.007041


Train Epoch: 7 [57600/60032 (96%)]	Loss: 0.221846
Train Epoch: 7 [58240/60032 (97%)]	Loss: 0.014029
Train Epoch: 7 [58880/60032 (98%)]	Loss: 0.100463
Train Epoch: 7 [59520/60032 (99%)]	Loss: 0.047687

Test set: Average loss: 0.0000, Accuracy: 9851/10000 (99%)

Train Epoch: 8 [0/60032 (0%)]	Loss: 0.053752
Train Epoch: 8 [640/60032 (1%)]	Loss: 0.004248
Train Epoch: 8 [1280/60032 (2%)]	Loss: 0.007790
Train Epoch: 8 [1920/60032 (3%)]	Loss: 0.044958
Train Epoch: 8 [2560/60032 (4%)]	Loss: 0.001204
Train Epoch: 8 [3200/60032 (5%)]	Loss: 0.050965
Train Epoch: 8 [3840/60032 (6%)]	Loss: 0.075709
Train Epoch: 8 [4480/60032 (7%)]	Loss: 0.048470
Train Epoch: 8 [5120/60032 (9%)]	Loss: 0.082573
Train Epoch: 8 [5760/60032 (10%)]	Loss: 0.115793
Train Epoch: 8 [6400/60032 (11%)]	Loss: 0.012298
Train Epoch: 8 [7040/60032 (12%)]	Loss: 0.014983
Train Epoch: 8 [7680/60032 (13%)]	Loss: 0.016053
Train Epoch: 8 [8320/60032 (14%)]	Loss: 0.134600
Train Epoch: 8 [8960/60032 (15%)]	Loss: 0.007649
Train Epoch: 8 [9

Train Epoch: 9 [41600/60032 (69%)]	Loss: 0.009618
Train Epoch: 9 [42240/60032 (70%)]	Loss: 0.054172
Train Epoch: 9 [42880/60032 (71%)]	Loss: 0.017948
Train Epoch: 9 [43520/60032 (72%)]	Loss: 0.043156
Train Epoch: 9 [44160/60032 (74%)]	Loss: 0.006336
Train Epoch: 9 [44800/60032 (75%)]	Loss: 0.006354
Train Epoch: 9 [45440/60032 (76%)]	Loss: 0.063234
Train Epoch: 9 [46080/60032 (77%)]	Loss: 0.020687
Train Epoch: 9 [46720/60032 (78%)]	Loss: 0.011056
Train Epoch: 9 [47360/60032 (79%)]	Loss: 0.116945
Train Epoch: 9 [48000/60032 (80%)]	Loss: 0.012899
Train Epoch: 9 [48640/60032 (81%)]	Loss: 0.046951
Train Epoch: 9 [49280/60032 (82%)]	Loss: 0.043778
Train Epoch: 9 [49920/60032 (83%)]	Loss: 0.009360
Train Epoch: 9 [50560/60032 (84%)]	Loss: 0.010767
Train Epoch: 9 [51200/60032 (85%)]	Loss: 0.019597
Train Epoch: 9 [51840/60032 (86%)]	Loss: 0.076839
Train Epoch: 9 [52480/60032 (87%)]	Loss: 0.009633
Train Epoch: 9 [53120/60032 (88%)]	Loss: 0.025454
Train Epoch: 9 [53760/60032 (90%)]	Loss: 0.002705


## Well Done!

And voilà! We now are training a real world Learning model using Federated Learning! As you observed, we modified 10 lines of code to upgrade the official Pytorch example on MNIST to a real Federated Learning setting!

## Shortcomings of this Example

Of course, there are dozen of improvements we could think of. We would like the computation to operate in parallel on the workers, to update the central model every `n` batches only, to reduce the number of messages we use to communicate between workers, etc.

On the security side it still has some major shortcomings. Most notably, when we call `model.get()` and receive the updated model from Bob or Alice, we can actually learn a lot about Bob and Alice's training data by looking at their gradients. We could **average the gradient across multiple individuals before uploading it to the central server**, like we did in Part 4.


# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!


### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)